In [1]:
# GPT-4 dynamic evaluation

import pandas as pd
import json
from config import config
import openai
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv
load_dotenv()
import os
import random

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

folder_path = 'files'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

def load_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_perturbation(perturbations):
    category = random.choice(list(perturbations.keys()))
    perturbation = random.choice(list(perturbations[category].items()))
    return category, perturbation

perturbations = load_file(config.perturbations)
knowledgebase = load_file(config.knowledgebase)

client = OpenAI()
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

assistant = client.beta.assistants.create(
    name=f"{F_NAME} AI Dynamic Evaluator",
    instructions=config.INSTRUCTION,
    model=config.GPT_MODEL,
)
show_json(assistant)

# Utility functions
def read_csv(file_path):
    return pd.read_excel(file_path)

def process_data_for_gpt(data):
    prompts = []
    for _, row in data.iterrows():
        question = row['Question']
        prompt = f"Please try your best to answer the following question.:\n\n{question}"
        prompts.append(prompt)
    return prompts

def submit_message_and_create_run(assistant_id, prompt):
    thread = client.beta.threads.create() # If you replace this globally it appends all answers to the one before.
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id), thread

def wait_on_run_and_get_response(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    messages = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    return [m.content[0].text.value for m in messages if m.role == 'assistant']

def ask_gpt4(prompt, ASSISTANT_ID):
    run, thread = submit_message_and_create_run(ASSISTANT_ID, prompt)
    response = wait_on_run_and_get_response(run, thread)
    if isinstance(response, list):
        response = ' '.join(map(str, response))
    response = response.replace("\\\\n", "\\n")
    response = response.strip()
    print(response)
    return response

def process_question_with_gpt4(row, assistant_id):
    start_time = time.time()  # Capture start time
    original_question = row['Question']
    category = row.get('Category', 'Static')  # Default to 'Static' if not present
    if category != "Dynamic":
        # Dynamic question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        category, (perturbation, description) = get_random_perturbation(perturbations)
        perturbed_qn = f"{original_question}\nResponse: {first_response}\nChange in circumstances: {perturbation} - {description}\n What should change in the response?"
        perturbed_response = ask_gpt4(perturbed_qn, assistant_id)
        final_analysis_qn = f"Original Question: {original_question}\nOrig Response: {first_response} \nPerturbation ({category}): {perturbation} - {description}\n {perturbed_response}\nKnowledgebase Content: {knowledgebase}\n Now consider the knowlegebase, what else ought we to do?"
        final_analysis_response = ask_gpt4(final_analysis_qn, assistant_id)
    else:
        # Static question processing logic
        first_response = ask_gpt4(original_question, assistant_id)
        perturbed_qn = perturbed_response = final_analysis_qn = final_analysis_response = "n/a"

    end_time = time.time()  # Capture end time
    latency = (end_time - start_time)/3  # Calculate latency

    return {
        'Model': config.GPT_MODEL,
        'Question': original_question, 
        'Response': first_response, 
        'Perturbed Question': perturbed_qn, 
        'Perturbed Response': perturbed_response, 
        'Final Analysis Question': final_analysis_qn, 
        'Final Analysis Response': final_analysis_response,
        'Latency': latency,
        'Category': row['Category'],
        'Type': row['Type']
    }

# Modify DataFrame to include new columns
new_data_columns = ['Model', 'Question', 'Response', 'Perturbed Question', 'Perturbed Response', 'Final Analysis Question', 'Final Analysis Response', 'Latency', 'Category', 'Type']
results_df = pd.DataFrame(columns=new_data_columns)
data = read_csv(config.questions)
prompts = process_data_for_gpt(data)
ASSISTANT_ID = assistant.id

# Process each question
for index, row in data.iterrows():
    processed_info = process_question_with_gpt4(row, ASSISTANT_ID)
    results_df = results_df.append(processed_info, ignore_index=True)

# Save the results
results_df.to_excel(config.gpt4results_csv_path, index=False)


SyntaxError: invalid syntax (699531674.py, line 108)

In [6]:
# How to combine files together into one
import re
import config
import importlib
importlib.reload(config)
from config import config, reset_config
import pandas as pd
from difflib import SequenceMatcher
import json

INSTRUCTION = config.INSTRUCTION
F_NAME = config.F_NAME
config.set_mode("dynamic")

def clean_text(text):
    """
    Remove non-ASCII characters from the text.
    """
    return ''.join(char for char in text if char.isascii())

def create_combined_csv(original_csv_path, interim_csv_path, combined_csv_path):
    # Read the original and interim data
    original_data = pd.read_excel(original_csv_path) #, encoding='utf-8-sig'
    interim_data = pd.read_excel(interim_csv_path)

    # Combine the data
    combined_data = pd.concat([original_data, interim_data], ignore_index=True)

    # Save the combined data to a new CSV file
    combined_data.to_excel(combined_csv_path, index=False)

def merge_on_contains(big_df, small_df, big_col, small_col):
    # Lowercase and strip whitespace for more effective matching
    big_df[big_col] = big_df[big_col].str.lower().str.strip()
    small_df[small_col] = small_df[small_col].str.lower().str.strip()

    # Check if 'category' column exists in small_df
    if 'category' in small_df.columns:
        # Create a new column for the merged category in big_df
        big_df['category'] = ''

        # Iterate over the small dataframe and update the category in the big dataframe
        for _, row in small_df.iterrows():
            contains_mask = big_df[big_col].str.contains(row[small_col])
            big_df.loc[contains_mask, 'category'] = row['category']
    else:
        # Handle the case when 'category' column does not exist
        # For example, you can set a default category or leave it as it is
        big_df['category'] = 'default_category'  # or any other handling logic

    return big_df

create_combined_csv(config.llmresults_file_path, config.gpt4results_csv_path, config.results_file_path)

# Reading the files
questions_df = pd.read_excel(config.questions)
results_df = pd.read_excel(config.results_file_path)

# Ensure the total number of questions in results_grouped_by_model.xlsx is a multiple of the number in questions.xlsx
if len(results_df) % len(questions_df) != 0:
    print(len(results_df))
    print(len(questions_df))
    raise ValueError("The total number of questions in results_grouped_by_model.xlsx must be a multiple of the number in questions.xlsx.")

# Replace questions in results_grouped_df with those from questions_df
num_repetitions = len(results_df) // len(questions_df)
repeated_questions = pd.concat([questions_df['Question']] * num_repetitions, ignore_index=True)
results_df['Question'] = repeated_questions

# All info saved in one results file! 
# Save the modified DataFrame to a new Excel file
results_df.to_excel(config.results_file_path, index=False)  # Replace with your desired path

# Applying the merge_on_contains function
merged_df = merge_on_contains(results_df, questions_df, 'Question', 'Question')

# Sorting the DataFrame by the 'Question' column
sorted_df = results_df.sort_values(by=['Question'])

combined_df = sorted_df.fillna('')
# Save the combined data
combined_df.to_excel(config.combined_file_path, index=False)


/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_8599/3567630668.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  contains_mask = big_df[big_col].str.contains(row[small_col])


In [ ]:
# Archive the intermediate files
import os
import glob
from config import config
directory = 'files/'
archive_directory = os.path.join(directory, '#Archive')

# Create the #Archive directory if it doesn't exist
if not os.path.exists(archive_directory):
    os.makedirs(archive_directory)

# List all files that start with F_NAME and exclude the specified files
files_to_move = [f for f in glob.glob(f"{directory}/{config.F_NAME}_*") 
                 if '_model_rankings' not in f and '_llmeval_results' not in f and 'questions' not in f and '_results_grouped_by_question_' not in f and '_allresults_grouped_by_model_' not in f]

# Move the files to the #Archive folder
for file in files_to_move:
    os.rename(file, os.path.join(archive_directory, os.path.basename(file)))
    print(f"Moved file: {file} to {archive_directory}")
